In [153]:
#1.2?

class WeightedGraph:

    def __init__(self,nodes):
        self.graph=[]
        self.weights={}
        for node in range(nodes):
            self.graph.append([])

    def add_node(self,node):
        self.graph[node]=[]

    def add_edge(self, node1, node2, weight):
        if node2 not in self.graph[node1]:
            self.graph[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def get_weights(self, node1, node2):
        if self.are_connected(node1, node2):
            return self.weights[(node1, node2)]

    def are_connected(self, node1, node2):
        for neighbour in self.graph[node1]:
            if neighbour == node2:
                return True
        return False

    def get_neighbors(self, node):
        return self.graph[node]

    def get_number_of_nodes(self,):
        return len(self.graph)
    
    def get_nodes(self,):
        return [i for i in range(len(self.graph))]

In [154]:
def BellmanFord(g,src,k):
    dist = {}
    path = {}
    num_relax = {}
    
    #initializing distance of all nodes to infinity and dist of 1st node as 0
    for i in range(g.get_number_of_nodes()):
        dist[i] = float("inf")
        path[i] = [src]

        num_relax[i] = 0
        
    dist[src] = 0
    path[src] = [src]

    
    for _ in range(len(g.get_nodes()) - 1):
        difference = False
        for node in g.get_nodes():
            for n in g.get_neighbors(node):
                weight = g.get_weights(node,n)
                if weight != None:
                    if dist[node] + weight < dist[n] and num_relax[node] < k:
                        dist[n] = dist[node] + weight
                        path[n] = path[node] + [n]
                        num_relax[n] += 1
                        difference = True
        if not difference:
            return dist,path


In [155]:

#FUNCTIONS TAKEN FROM LECTURE NOTES TO BUILD UPON
import math

class MinHeap:
    def __init__(self, data):
        self.items = data
        self.length = len(data)
        
        self.map = {}
        for i in range(self.length):
            self.map[self.items[i].value] = i
            
        self.build_heap()

    def find_left_index(self,index):
        return 2 * (index + 1) - 1

    def find_right_index(self,index):
        return 2 * (index + 1)

    def find_parent_index(self,index):
        return (index + 1) // 2 - 1  

    def sink_down(self, index):
        smallest_known_index = index
        left_index = self.find_left_index(index)
        right_index = self.find_right_index(index)
        if left_index < self.length and self.items[left_index].key < self.items[smallest_known_index].key:
            smallest_known_index = left_index

        if right_index < self.length and self.items[right_index].key < self.items[smallest_known_index].key:
            smallest_known_index = right_index

        if smallest_known_index != index:
            self.items[index], self.items[smallest_known_index] = self.items[smallest_known_index], self.items[index]
            
            self.map[self.items[index].value] = index
            self.map[self.items[smallest_known_index].value] = smallest_known_index

            self.sink_down(smallest_known_index)

    def build_heap(self):
        for i in range(self.length // 2 - 1, -1, -1):
            self.sink_down(i) 

    def insert(self, node):
        self.items.append(node)
        self.map[node.value] = self.length
        self.length += 1
        self.swim_up(self.length - 1)

    def swim_up(self, index):
        while index > 0 and self.items[index].key < self.items[self.find_parent_index(index)].key:
            parent_index = self.find_parent_index(index)
            self.items[index], self.items[parent_index] = self.items[parent_index], self.items[index]
            #update map
            self.map[self.items[index].value] = index
            self.map[self.items[parent_index].value] = parent_index
            index = parent_index

    def extract_min(self):
        if self.length == 0:
            return None
        self.items[0], self.items[self.length - 1] = self.items[self.length - 1], self.items[0]
        self.map[self.items[self.length - 1].value] = self.length - 1
        self.map[self.items[0].value] = 0

        min_node = self.items.pop()
        self.length -= 1
        self.map.pop(min_node.value, None)
        if self.length > 0:
            self.sink_down(0)
        return min_node

    def decrease_key(self, value, new_key):
        if value in self.map and new_key < self.items[self.map[value]].key:
            index = self.map[value]
            self.items[index].key = new_key
            self.swim_up(index)

    def is_empty(self):
        return self.length == 0

class Item:
    def __init__(self, value, key):
        self.key = key
        self.value = value
    
    def __str__(self):
        return "(" + str(self.key) + "," + str(self.value) + ")"

In [156]:
class BellmanFord_no_k:

    def __init__(self,g,src):
        self.g = g
        self.src = src
        self.dist = {}
        self.path = {}
        
    def startvals(self):
        for i in range(self.g.get_number_of_nodes()):
            self.dist[i] = float("inf")
            self.path[i] = [self.src]
            
        self.dist[self.src] = 0
        self.path[self.src] = [self.src]
    
    def code(self):
        
        for _ in range(len(self.g.get_nodes()) - 1):
            difference = False
            for node in self.g.get_nodes():
                for n in self.g.get_neighbors(node):
                    weight = self.g.get_weights(node,n)
                    if weight != None:
                        if self.dist[node] + weight < self.dist[n]:
                            self.dist[n] = self.dist[node] + weight
                            self.path[n] = self.path[node] + [n]
                            difference = True
            if not difference:
                return self.dist,self.path
            
    def get_distance(self):
        return self.dist
    
    def get_path(self):
        return self.path


In [157]:
class Dijkstra_no_k:
    def __init__(self, graph):
        self.graph = graph
        self.node_count = graph.get_number_of_nodes()
        self.distances = {node: float('inf') for node in self.graph.get_nodes()}
        self.paths = {node: [] for node in range(self.node_count)}
    
    def find_shortest_paths(self, source):
        self.distances[source] = 0
        self.paths[source] = [source]
        
        initialize_heap = [Item(node, self.distances[node]) for node in range(self.node_count)]
        self.min_heap = MinHeap(initialize_heap)
            
        while not self.min_heap.is_empty():
            current_node = self.min_heap.extract_min()
            node_index = int(current_node.value)
            
            if self.distances[node_index] == float('inf'):
                continue
            
            for neighbor in self.graph.get_neighbors(node_index):
                weight = self.graph.get_weights(node_index, neighbor)
                new_distance = self.distances[node_index] + weight
                
                if new_distance < self.distances[neighbor]:
                    self.distances[neighbor] = new_distance
                    self.paths[neighbor] = self.paths[node_index] + [neighbor]
                    self.min_heap.decrease_key(neighbor, new_distance)
    
        return self.distances, self.paths


In [158]:
#2

def neg_wt(g):
    neg = False
    for node in g.get_nodes():
        for n in g.get_neighbors(node):
            wt = g.get_weights(node,n)
            if wt < 0:
                neg =  True
                break
        if neg == True:
            break
    return neg
    

class All_spa:
    
    def __init__(self,g):
        self.g = g
        self.dist = {}
        self.path = {}
    
    def aspa(self):
        if neg_wt(self.g) == True:
            for n in self.g.get_nodes():
                bellmang = BellmanFord_no_k(self.g,n)
                bellmang.startvals()
                bellmang.code()
                
                distn = bellmang.get_distance()
                pathn = bellmang.get_path()
                
                self.dist[n] = distn
                self.path[n] = {}
                for i in self.g.get_nodes():
                    self.path[n][i] = pathn[i][-1]
            print(self.path)
            print(self.dist)
                    
        else:
            for n in self.g.get_nodes():
                dijkstrag = Dijkstra_no_k(self.g)
                distn, pathn = dijkstrag.find_shortest_paths(n)
                self.dist[n] = distn
                self.path[n] = {}
                for i in self.g.get_nodes():
                    if pathn[i]:
                        self.path[n][i] = pathn[i][-1]
                    else:
                        self.path[n][i] = None
            print(self.path)
            print(self.dist)       
                    
    def distances(self,u,v):
        return self.dist[u][v]
    
    def previous(self,u,v):
        return self.path[u][v]

In [159]:
graph = WeightedGraph(5)
graph.add_edge(0, 1, 1)
graph.add_edge(1, 2, 1)
graph.add_edge(2, 0, 3)
graph.add_edge(2, 3, 2)
graph.add_edge(3, 4, 2)

gg = All_spa(graph)
gg.aspa()

print(gg.previous(0,3))
print(gg.distances(0,3))

{0: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}, 1: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}, 2: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}, 3: {0: None, 1: None, 2: None, 3: 3, 4: 4}, 4: {0: None, 1: None, 2: None, 3: None, 4: 4}}
{0: {0: 0, 1: 1, 2: 2, 3: 4, 4: 6}, 1: {0: 4, 1: 0, 2: 1, 3: 3, 4: 5}, 2: {0: 3, 1: 4, 2: 0, 3: 2, 4: 4}, 3: {0: inf, 1: inf, 2: inf, 3: 0, 4: 2}, 4: {0: inf, 1: inf, 2: inf, 3: inf, 4: 0}}
3
4
